In [1]:
import os
import uuid
import glob 
import pickle
import numpy as np
from tqdm import tqdm
from PIL import Image
from maskrcnn_benchmark.config import cfg
import matplotlib.pyplot as plt
from maskrcnn_benchmark.modeling.detector import build_detection_model

%load_ext autoreload
%autoreload 2

In [13]:
cfg.merge_from_file('configs/garment_detection-1gpu.yaml')
cfg.merge_from_list(['MODEL.WEIGHT', '/data/logs/garment_detection/imaterialist+modanet_continue/model_45000.pth'])
cfg.merge_from_list(['DATALOADER.ASPECT_RATIO_GROUPING', 'False'])
device = torch.device(cfg.MODEL.DEVICE)
cfg_ = cfg.clone()

model = build_detection_model(cfg_)
model.to(device)

In [19]:
from maskrcnn_benchmark.data import make_data_loader

In [44]:
data_loader = make_data_loader(
    cfg,
    is_train=True,
    is_distributed=False,
)

When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14


In [45]:
data_loader_iter = iter(data_loader)

tensor([[[152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         ...,
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199]],

        [[139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         ...,
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535]],

        [[132.2283, 132.2283, 132.2283,  ...

         [132.2283, 132.2283, 132.2283,  ..., 132.2283, 132.2283, 132.2283]]])tensor([[[152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         ...,
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199]],

        [[139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         ...,
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ...,

         [132.2283, 132.2283, 132.2283,  ..., 132.2283, 132.2283, 132.2283]]])
tensor([[[152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         ...,
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199],
         [152.0199, 152.0199, 152.0199,  ..., 152.0199, 152.0199, 152.0199]],

        [[139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         ...,
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ..., 139.0535, 139.0535, 139.0535],
         [139.0535, 139.0535, 139.0535,  ...

         [132.2283, 132.2283, 132.2283,  ..., 132.2283, 132.2283, 132.2283]]])


In [46]:
batch = next(data_loader_iter)

In [43]:
batch[0]